<a href="https://colab.research.google.com/github/kytk/AI-MAILs/blob/main/python_7_scikit-learn-2.ipynb?hl=ja" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 医療従事者のためのPython: 機械学習 (2)

根本清貴 (筑波大学医学医療系精神医学)

Ver.20240823

### 目次
1. 機械学習とは (復習)
2. 機械学習の適切なモデルの選び方
3. 教師なし学習の概要
4. 主成分分析
5. 乳がんデータセットでの主成分分析
6. k-meansクラスタリング
7. 乳がんデータセットでのk-meansクラスタリング

### 1. 機械学習とは (復習)
- 機械学習の定義
    - 「データからルールやパターンを導き出し、予測や意思決定を行う技術」
    - (従来: 人間がルールを決める) 
- 機械学習の種類
    - 教師あり学習
        - 入力データとその対応する正解(ラベル)がペアになったデータセットを用いてモデルを訓練
        - 新しいデータが入ってきた時にそのモデルから正しいラベルを予測
        - 例: 画像の分類(犬と猫の画像を分類)、スパムメールの分類、価格予測
    - 教師なし学習
        - ラベルのないデータを用いてモデルを訓練
        - データの内部構造やパターンを見つけ出す
        - 例: クラスタリング(似たデータをグループに分ける)、次元削減(データの特徴を少数の重要な特徴に圧縮する)
    - 強化学習
        - エージェント(学習者)が環境と相互作用しながら学習する。エージェントは行動を選択し、その結果として得られる報酬を基に次の行動を改善する
        - 長期的な累積報酬を最大化することが目標
        - 例: ゲーム、ロボット制御、自動運転
- 本日は、教師なし学習 (unsupervised learning) について学ぶ


### 2. 機械学習の適切なモデルの選び方

- scikit-learn のホームページにわかりやすい図が示されている
    - https://scikit-learn.org/1.3/tutorial/machine_learning_map/index.html
<img src="https://scikit-learn.org/1.3/_static/ml_map.png">
- これを見ると、機械学習を行うには、データは最低50例必要であることがわかる
    - 根拠を探したが見つけられなかった

### 3. 教師なし学習の概要

- データに対する正解ラベルを必要とせず、データの内部構造やパターンを学習する方法
- ラベルのないデータを用いて、データの特性を理解し、データを構造化することが目的


#### 3.1. 教師なし学習の主要な手法

1. **次元削減 (Dimensionality Reduction)**
   - データの特徴量の次元を減らし、データを簡潔に表現する手法
   - 主なアルゴリズム: 主成分分析 (PCA)、独立成分分析 (ICA)、t-SNEなど
   - **例**: 遺伝子発現解析において、数千の遺伝子データを主成分分析(PCA)で数十の主要パターンに圧縮することで、疾患の特徴や患者群の違いを効率的に可視化する

2. **クラスタリング (Clustering)**
   - データを似た特徴を持つグループ（クラスター）に分ける手法
   - 主なアルゴリズム: k-means、階層型クラスタリング、DBSCANなど
   - **例**: 患者の血液検査結果を用いて、似た特徴を持つ患者群を自動的に分類する。k-meansアルゴリズムを使用して、例えば3つのクラスターに分けると、「健康な患者群」「代謝異常リスク群」「炎症性疾患リスク群」といった意味のあるグループが形成され、それぞれに適した予防策や治療方針の策定に役立つ


3. **異常検知 (Anomaly Detection)**
   - 正常なデータから外れる異常なデータポイントを検出する手法。
   - 主なアルゴリズム: 一クラスSVM、孤立森林、ガウス混合モデルなど。
   - **例**: 心電図データに孤立森林アルゴリズムを適用することで、通常の心拍パターンから逸脱した不整脈や心臓異常を自動的に検出し、早期診断や緊急対応につなげることができる

- 今回は主成分分析とk-meansクラスタリングを実装する
   


### 4. 主成分分析
- 次元削減のうち、よく用いられている主成分分析をscikit-learnで実装する
- 医療データに応用する前に、わかりやすいように学生の5教科 (5次元) の試験結果を主成分分析で2次元にしてみる

#### 4.1. パッケージのインポート
- NumPy
- Pandas
- Matplotlib
- Seaborn
- Scikit-learn の preprocessingモジュール から 標準化関数 StandardScaler
- Scikit-learn の decompositionモジュール から PCA


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#### 4.2. データの読み込み

- student_scores.csv をダウンロード
- Pandas で df として読み込む


In [ ]:
# コマンドの前に ! をつけると、Linuxコマンドが動作できる
![[ -f student_scores.csv ]] || wget https://raw.githubusercontent.com/kytk/AI-MAILs/main/data/student_scores.csv

In [ ]:
# student_scores.csv を df という名前の Pandasデータフレームとして読み込み
df = pd.read_csv('student_scores.csv')

In [ ]:
# データを表示
# group は science が理系、humanities が文系
# 150人の5教科のテストの成績
df

#### 4.3. PandasのDataFrameをNumpyのndarrayに変換

- scikit-learn で扱うデータは基本、NumPy配列
- 前回はサンプルデータを使っているのであまり意識しなかったが、PandasのDataFrameをNumPyに変換する
- PandasからNumPyへの変換は、`to_numpy()` メソッドを使うだけなのでとても簡単
- group以外の5教科の点数をNumPy配列に変換する

In [ ]:
# dfから 'group' の列だけ drop して、df_scoresとして代入
# df.drop('group', axis=1) で削除できる
df_scores = df.drop('group', axis=1)

# df_scores を確認
df_scores

In [ ]:
# PandasのDataFrame df_scores を NumPy配列に変換
# to_numpy() メソッドを使用
scores = df_scores.to_numpy()

# scores の最初の10人を確認
# scores[0:10,:] だが、後半の : は省略できる
scores[0:10]

#### 4.3. データの前処理: 標準化
- 5教科を 平均値0, 標準偏差1 となるように標準化する
- `StandardScaler()` 関数を使用する

In [ ]:
# データの標準化

# StandardScaler() 関数を使用して標準化を行うオブジェクト scaler を生成
scaler = StandardScaler()  
# 標準化するためのパラメータを計算(fit)し、適用(transform)する
scores_standardized = scaler.fit_transform(scores)

# 最初の数行を確認
# scores, scores_scaled は numpy型 なので、scores_scaled[:3] で最初の3行が表示される

# scores
print('scores')
print(scores[:3])

# scores_standardized
# np.round(X,2)はXの結果を小数点2位で丸めている
print('\nscores_standardized')
print(np.round(scores_standardized[:3],2))


In [ ]:
# 平均と標準偏差を確認
# axis=0 とすることで、各列における行の平均や標準偏差を求められる
print('\nscoresの平均:', np.round(scores.mean(axis=0),1))
print('scoresの標準偏差:', np.round(scores.std(axis=0),1))

print('\nscores_standardizedの平均:', np.round(scores_standardized.mean(axis=0),2))
print('scores_standardizedの標準偏差:', np.round(scores_standardized.std(axis=0),2))

#### 4.4. 主成分分析の理解

- 主成分分析の本質は、「データの分散が一番大きくなるような軸を見つける」こと
- 以下のサイトが理解に役立つ
  - https://www.billconnelly.net/?p=697
- 5教科だと図示しづらいため、5教科ではなく、3教科(英語、数学、国語)の試験結果を2次元に圧縮してみる
- 最初に3教科の試験結果を3次元で表示する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.animation as animation

# 前処理データから、英語、数学、国語の結果を抽出
X = scores_standardized[:,[0,1,3]]
X = scaler.fit_transform(X)

# PCA実行 (この後説明するので、今は理解できなくていい)
pca3 = PCA(n_components=2)
X_pca = pca3.fit_transform(X)

# 以下は、図示のためのコード
# 3D散布図（元のデータ）
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')
scatter1 = ax1.scatter(X[:, 0], X[:, 1], X[:, 2],
                     c=df['group'].map({'science': 0, 'humanities': 1}), 
                     cmap='winter')
ax1.set_xlabel('English')
ax1.set_ylabel('Math')
ax1.set_zlabel('Japanese')
ax1.set_title('3D Data with PCA Plane')

# データの範囲を取得
x_min, x_max = ax1.get_xlim()
y_min, y_max = ax1.get_ylim()
z_min, z_max = ax1.get_zlim()

# 原点を通る点線を追加
ax1.plot([x_min, x_max], [0, 0], [0, 0], 'k--', alpha=0.5)  # x軸
ax1.plot([0, 0], [y_min, y_max], [0, 0], 'k--', alpha=0.5)  # y軸
ax1.plot([0, 0], [0, 0], [z_min, z_max], 'k--', alpha=0.5)  # z軸

# カラーバーの追加
plt.colorbar(scatter1, ax=ax1, label='Group (0: Science, 1: Humanities)')

# 視点の調整
ax1.view_init(elev=15, azim=135)  # 仰角15度、方位角135度

plt.tight_layout()
plt.show()


- この空間に線を一本ひく。その線は、データが一番ばらついて見える線であり、それが第1主成分の軸
- その次に、第1主成分に垂直に線をひく。その線もできるだけデータがばらつくようにする。これが第2主成分の軸
- これらの軸を構成する単位ベクトルを **固有ベクトル** という。固有ベクトルの大きさは1
    - 下の図では、ベクトルを見やすくするために、固有ベクトルを3.5倍している
- そのベクトルにおけるデータの分散を **固有値** という

In [ ]:
#### 先程のプロットと同じ内容
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')
scatter1 = ax1.scatter(X[:, 0], X[:, 1], X[:, 2],
                     c=df['group'].map({'science': 0, 'humanities': 1}), 
                     cmap='winter')
ax1.set_xlabel('English')
ax1.set_ylabel('Math')
ax1.set_zlabel('Japanese')
ax1.set_title('3D Data with PCA Plane')

x_min, x_max = ax1.get_xlim()
y_min, y_max = ax1.get_ylim()
z_min, z_max = ax1.get_zlim()

ax1.plot([x_min, x_max], [0, 0], [0, 0], 'k--', alpha=0.5)  # x軸
ax1.plot([0, 0], [y_min, y_max], [0, 0], 'k--', alpha=0.5)  # y軸
ax1.plot([0, 0], [0, 0], [z_min, z_max], 'k--', alpha=0.5)  # z軸
#### ここまで

# 主成分ベクトルの描画
for i, color in enumerate(['r', 'g']):
    ax1.quiver(pca3.mean_[0], pca3.mean_[1], pca3.mean_[2],
              pca3.components_[i, 0], pca3.components_[i, 1], pca3.components_[i, 2],
              length=3.5, color=color, label=f'PC{i+1}')

ax1.legend()

# カラーバーの追加
plt.colorbar(scatter1, ax=ax1, label='Group (0: Science, 1: Humanities)')

# 視点の調整
ax1.view_init(elev=15, azim=135)  # 仰角15度、方位角135度

plt.tight_layout()
plt.show()

- 第1主成分と第2主成分で構成される平面を図示する

In [ ]:
#### 先程のプロットと同じ内容
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')
scatter1 = ax1.scatter(X[:, 0], X[:, 1], X[:, 2],
                     c=df['group'].map({'science': 0, 'humanities': 1}), 
                     cmap='winter')
ax1.set_xlabel('English')
ax1.set_ylabel('Math')
ax1.set_zlabel('Japanese')
ax1.set_title('3D Data with PCA Plane')

x_min, x_max = ax1.get_xlim()
y_min, y_max = ax1.get_ylim()
z_min, z_max = ax1.get_zlim()

ax1.plot([x_min, x_max], [0, 0], [0, 0], 'k--', alpha=0.5)  # x軸
ax1.plot([0, 0], [y_min, y_max], [0, 0], 'k--', alpha=0.5)  # y軸
ax1.plot([0, 0], [0, 0], [z_min, z_max], 'k--', alpha=0.5)  # z軸

for i, color in enumerate(['r', 'g']):
    ax1.quiver(pca3.mean_[0], pca3.mean_[1], pca3.mean_[2],
              pca3.components_[i, 0], pca3.components_[i, 1], pca3.components_[i, 2],
              length=3.5, color=color, label=f'PC{i+1}')
#### ここまで

# PCA平面の描画
# 平面の角を計算
corner00 = pca3.mean_
corner01 = corner00 + pca3.components_[0] * 3.5
corner10 = corner00 + pca3.components_[1] * 3.5
corner11 = corner00 + pca3.components_[0] * 3.5 + pca3.components_[1] * 3.5

# 平面を描画
ax1.plot_surface(
    X=np.array([[corner00[0], corner01[0]], [corner10[0], corner11[0]]]),
    Y=np.array([[corner00[1], corner01[1]], [corner10[1], corner11[1]]]),
    Z=np.array([[corner00[2], corner01[2]], [corner10[2], corner11[2]]]),
    alpha=0.2, color='r'
)

ax1.legend()

# カラーバーの追加
plt.colorbar(scatter1, ax=ax1, label='Group (0: Science, 1: Humanities)')

# 視点の調整
ax1.view_init(elev=15, azim=135)  # 仰角15度、方位角135度

plt.tight_layout()
plt.show()


### 参考: 以下は、matplotlib の結果を保存したい場合やアニメーションにしたい場合のコード

## グラフを保存したい場合
#fig1.savefig('3d_with_PCA.png')

## アニメーションの作成
#def animate(frame):
# ax1.view_init(elev=15, azim=frame)
# return fig1,

#ani = animation.FuncAnimation(fig1, animate, frames=np.arange(0, 360, 2), interval=100)

## アニメーションの表示
#from IPython.display import HTML
#HTML(ani.to_jshtml())

## アニメーションをファイルに保存
#ani.save('rotating_3d_plot.gif', writer='pillow')

- 上記の3次元グラフを回転させることで、よりイメージがつかみやすくなる (金子貴久子先生提供)
    <img src = "https://www.nemotos.net/nb/img/rotating_3d_plot.gif">

- 元のデータを、第1主成分、第2主成分で構成される平面に投影する

In [ ]:
# 2D散布図（PCAの第1主成分、第2主成分で構成される座標軸）
fig2 = plt.figure(figsize=(6, 4))
ax2 = fig2.add_subplot(111)
scatter2 = ax2.scatter(X_pca[:, 0], X_pca[:, 1],
                       c=df['group'].map({'science': 0, 'humanities': 1}), cmap='winter')

ax2.set_xlabel('First Principal Component')
ax2.set_ylabel('Second Principal Component')
ax2.set_title('2D Projection after PCA')
ax2.set_xlim(-3.5,3.5)
ax2.set_ylim(-3.5,3.5)

# カラーバーの追加
plt.colorbar(scatter2, ax=ax2, label='Group (0: Science, 1: Humanities)')

plt.tight_layout()
plt.show()


- データの第1主成分のばらつきは-3〜3ぐらいにわたる
- 第2主成分のばらつきは、-2.5〜2.5程度で第1主成分ほどばらついていない
- このように、3次元のデータをデータのばらつきに基づいて、2次元で表示できた

#### 4.5. 5教科の試験結果に対して、PCAを適用し、2次元に削減
- Scikit-learn の PCAの実行そのものは非常に簡単
- `PCA()` 関数を使用して PCAを行うオブジェクトを生成する
    - その際、`n_components=2` のように次元を指定する
    - ここでは 2 に設定しているので、データを2次元に圧縮する
    - つまり、元のデータから、もっとも重要な2つの主成分だけを保持するようにPCAに指示する
- その後、準備したオブジェクトのメソッド `fit_transform()` にデータを投入する
    - `fit()`: 入力データに基づいて、固有ベクトルを見つけ、それぞれのベクトルにおけるデータの分散 (固有値) を計算する
    - `transform()`: 学習したモデルを使って、入力データを主成分空間に変換する
    - `fit_transform()`: `fit()` と `transform()` を同時に行う
- scores_pca は、元のデータを2次元の主成分空間に投影した結果
    - 各行は元のデータに対応し、2つの列はそれぞれ第1主成分と第2主成分の値を意味する



In [ ]:
# PCAの適用

# 5教科のスコアに対してPCAを行うオブジェクト pca5 を生成
pca5 = PCA(n_components=2)

# 前処理が終わったデータを投入し、fitで学習、transformで変換
scores_pca = pca5.fit_transform(scores_standardized) 

##### 4.5.1. PCAの結果を確認
- PCAの結果は、上記の `pca5` と `scores_pca` の2つのオブジェクトを確認する必要がある


##### scores_pca (PCAを行った結果がおさめられているオブジェクト) 
- scores_pca: PCAの結果得られた各個人の主成分(今の場合は2)からできる座標軸における値
  - scores_pcaの形状: `scores_pca.shape`
  - scores_pca の実際の値 `scores_pca`
    - 第1主成分をx軸、第2主成分をy軸とした時の座標というイメージ


In [ ]:
# 大きさは150行2列
# 150人の5教科のデータが2つの情報に集約された
print('scoresの形状: ', scores_pca.shape)

In [ ]:
# scores_pca そのものを見る
# 各データが新たな第1軸, 第2軸に対する座標で表現されている
print('scores_pca の実際の値: 5人分\n', np.round(scores_pca[:5],2))

##### pca5 (PCAを行うために生成したオブジェクト) の属性
- このオブジェクトの属性にPCAを行ったことにより得られた主成分の固有ベクトル、固有値、寄与率などがおさめられている
  - 各主成分の元の特徴量空間 (今の場合は5次元) における固有ベクトル: `pca5.components_`
  - 各固有ベクトルにおける固有値 (元データの分散): `pca5.explained_variance_`
  - 寄与率 (元データの分散を1とした時、各主成分の固有値がその何割を説明するか: `pca5.explained_variance_ratio_`
  - 累積寄与率: 主成分でデータの何％を説明できるか(累積寄与率): `np.sum(pca5.explained_variance_ratio_)`

- `pca5.components_`
   - 各主成分の固有ベクトルを表す行列
   - 各行が主成分(第1主成分、第2主成分)に対応し、列は元の特徴量(教科)に対応

In [ ]:
# 固有ベクトルは、元の特徴量空間において、第1軸、第2軸がどの方向を向いているかを示す
print('固有ベクトル(主成分の方向): \n', np.round(pca5.components_,2))

- `pca5.components_.T`
   - `.T`は転置を意味
   - 転置することで、各行が元の特徴量(教科)に、各列が主成分(第1主成分、第2主成分)に対応

In [ ]:
print('固有ベクトルの転置: \n', np.round(pca5.components_.T,2))

- `pca5.explained_variance_`
   - 固有値 (各主成分における元データの分散)
   - 各主成分がデータの全体の分散のうちどれだけを説明しているかを意味
   - 固有値は、データ分析や次元削減を行う際に、どの主成分を重視すべきかを判断する上で重要な指標となる
- `np.sqrt(pca5.explained_variance_)`
   - 分散の平方根を取ることで、固有値を標準偏差のスケールに変換

In [ ]:
print('固有値(主成分の分散): ', np.round(pca5.explained_variance_,2))
print('固有値の標準偏差: ', np.round(np.sqrt(pca5.explained_variance_),2))

- 大きい固有値は以下のように解釈できる
    1. データの分散を多く捉えている: 固有値が大きいほど、その主成分は元のデータの分散（ばらつき）をより多く表現している
    2. 情報量が多い: 大きな固有値を持つ主成分は、元のデータセットの中でより多くの情報を保持している
    3. 特徴の重要性が高い: 固有値が大きいほど、その主成分が表す特徴がデータ全体の構造を理解する上で重要である
    4. データの変動パターンをよく表現している: 大きな固有値を持つ主成分は、データ内の主要な変動パターンやトレンドをより正確に捉えている
    5. 次元圧縮時の情報保持率が高い: 固有値が大きいほど、その主成分を使ってデータを低次元に圧縮した際に、元のデータの特性をより多く保持できている
    6. データの構造をよく反映している: 大きな固有値を持つ主成分は、データセット内の重要な構造や関係性をより良く表現している

- `pca5.explained_variance_ratio_`
  - 固有値の寄与率
    - 各主成分がデータの全体的な変動（分散）のうち、どれだけの割合を説明しているかを示す
    - 値は0から1の間で、全ての値の合計は1（つまり100%）になる
    - 大きい値ほど、その主成分がデータの構造をよく捉えていることを意味する
    - 最初の数個の主成分の寄与率が高い場合、少数の主成分でデータの大部分を表現できることを示す
- `sum(pca5.explained_variance_ratio_)`
  - 累積寄与率
    - 主成分全体でデータの何％を説明できるかを示す


In [ ]:
print('固有値の寄与率: ', np.round(pca5.explained_variance_ratio_,2))
print('累積寄与率: ', np.round(sum(pca5.explained_variance_ratio_),2))

#### 4.5.2. 主成分負荷量

- 主成分負荷量とは
    - 主成分の固有ベクトルに主成分の固有値の標準偏差をかけ合わせたもの
    - 元の特徴量が各主成分にどの程度寄与しているかを示す
    - -1から1の間の値をとり、絶対値が大きいほど寄与度が高い
- 主成分負荷量の解釈
    - 正の値: その特徴量が主成分に正の影響を与える
    - 負の値: その特徴量が主成分に負の影響を与える
    - 絶対値が大きい: その特徴量がその主成分に強く影響している    

In [ ]:
# 各特徴量（教科）の主成分への寄与度 (主成分負荷量)

# pca5.components_ を列で示したいので、.T で転置する
# それに固有値の標準偏差をかけあわせる
loadings = np.round(pca5.components_.T * np.sqrt(pca5.explained_variance_),2)

# 結果を表にしたいので、Pandasを使って表にする
# NumPy配列をPandasにするときには、pd.DataFrame(numpy配列, columns=[列名], index=[行名])とする
loadings_df = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=df.columns[:-1])


print("\n各特徴量の主成分負荷量:")
loadings_df

- 主成分負荷量からの考察
  - 英語は PC2 の負の方向に大きく寄与している
  - 数学と理科は PC1 の負の方向に寄与している
  - 国語は PC1 と PC2 に同程度寄与している
  - 社会は PC1 の正の方向に寄与している

- 第1主成分は、プラスなら文系、マイナスなら理系ということで、「文理軸」と言えるか
- 第2主成分は、プラスなら国語、マイナスなら英語ということで、「言語軸」と言えるか

#### 4.6. PCAの結果の可視化

##### 4.6.1. 散布図
- scores_pca を横軸 第1主成分 PC1, 縦軸 第2主成分 PC2 の座標にプロット
- 個々の点は個人を示す
- 色は、その人が理系か文系を示す

In [ ]:
# 散布図にプロット
# 横軸に scores_pcaの第1列、縦軸に scores_pcaの第2列をプロット

# 理系の人を0(青色), 文系の人を1(緑色)で表示

plt.figure(figsize=(8, 6))
scatter = plt.scatter(scores_pca[:, 0], scores_pca[:, 1], 
                      c=df['group'].map({'science': 0, 'humanities': 1}), cmap='winter')
plt.colorbar()
plt.xlabel('First Principal Component')
plt.ylabel('Second  Principal Component')
plt.title('PCA results')
plt.show()


- 以下のコードで、どの点がどの個人かを同定することもできる

In [ ]:
plt.figure(figsize=(8, 6))
scatter = plt.scatter(scores_pca[:, 0], scores_pca[:, 1], 
                      c=df['group'].map({'science': 0, 'humanities': 1}), cmap='winter')
plt.colorbar()
plt.xlabel('First Principal Component')
plt.ylabel('Second  Principal Component')
plt.title('Student Scores in PC Space')
for i, txt in enumerate(df.index):
    plt.annotate(txt, (scores_pca[i, 0], scores_pca[i, 1]))
plt.show()

#### 4.6.2. 主成分平面における各教科の位置
- 固有ベクトルは、各主成分において元の特徴量がどの程度寄与するかを意味しているが、見方を変えて、「各教科が主成分にどのように寄与しているか」という観点で見ると、各教科が第1主成分、第2主成分にどう影響しているかを見ることができる
- 各教科が、第1主成分、第2主成分を軸とする空間の中でどこに位置するかを可視化する

In [ ]:
# 固有ベクトルを改めて表示
print("固有ベクトル:")
print(pd.DataFrame(np.round(pca5.components_.T,2), 
                   columns=['PC1', 'PC2'], 
                   index=df.columns[:-1]))

# 各教科のPC1, PC2の値からなるベクトルをプロット
plt.figure(figsize=(6, 6))
for i, (x, y) in enumerate(zip(pca5.components_[0], pca5.components_[1])):
    plt.arrow(0, 0, x, y, head_width=0.05, head_length=0.05, fc='r', ec='r')
    plt.text(x*1.2, y*1.1, df.columns[i], fontsize=12)

# 現在のグラフに原点を通る水平線 (x軸, y軸) を追加
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.5)

plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Variables factor map (PCA)')
plt.show()

- これから改めて以下が考察される
    - PC1 は文系か理系かを示す。値が正ならば文系、負ならば理系
    - PC2 は言語を示す。値が正ならば国語がより得意、負ならば英語が得意

### 5. 乳がんデータセットを用いた主成分分析
- 乳がんデータセットの30の特徴量を2次元まで削減する
- その2次元がもしかしたら良性・悪性の特徴に近いかもしれないので、その一致度も見てみる

#### 5.0. 乳がんデータセット (復習)
- データセットの名称: Breast Cancer Wisconsin (Diagnostic) dataset
- サンプル数: 569
- 特徴量の数: 30
- ターゲットの種類: 2クラス（良性と悪性）
  - 0: 悪性; 1: 良性
- 特徴量の種類: 実数値


- 細胞診における細胞核の30の特徴量

| 英語 | 日本語 | 英語 | 日本語 |
| --- | --- | --- | --- |
| mean radius | 平均半径 | mean texture | 平均テクスチャ |
| mean perimeter | 平均周囲長 | mean area | 平均面積 |
| mean smoothness | 平均平滑度 | mean compactness | 平均コンパクト度 |
| mean concavity | 平均陥凹度 | mean concave points | 平均陥凹点数 |
| mean symmetry | 平均対称性 | mean fractal dimension | 平均フラクタル次元 |
| radius error | 半径誤差 | texture error | テクスチャ誤差 |
| perimeter error | 周囲長誤差 | area error | 面積誤差 |
| smoothness error | 平滑度誤差 | compactness error | コンパクト度誤差 |
| concavity error | 陥凹度誤差 | concave points error | 陥凹点数誤差 |
| symmetry error | 対称性誤差 | fractal dimension error | フラクタル次元誤差 |
| worst radius | 最悪の半径 | worst texture | 最悪のテクスチャ |
| worst perimeter | 最悪の周囲長 | worst area | 最悪の面積 |
| worst smoothness | 最悪の平滑度 | worst compactness | 最悪のコンパクト度 |
| worst concavity | 最悪の陥凹度 | worst concave points | 最悪の陥凹点数 |
| worst symmetry | 最悪の対称性 | worst fractal dimension | 最悪のフラクタル次元 |


#### 5.1. パッケージのインポート
- NumPy
- Pandas
- Matplotlib
- Seaborn
- Scikit-learn の preprocessingモジュール から 正規化関数 MinMaxScaler
- Scikit-learn の decompositionモジュール から PCA


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#### 5.2. データの読み込み

- 先程と同じように、サンプルデータセットを読み込むのではなく、Excelファイルを Pandas に読み込む
- target は不要のために外す

In [ ]:
# コマンドの前に ! をつけると、Linuxコマンドが動作できる
![[ -f breast_cancer_data.xlsx ]] || wget https://raw.githubusercontent.com/kytk/AI-MAILs/main/data/breast_cancer_data.xlsx

- Pandas の DataFrame への読み込み

In [ ]:
df = pd.read_excel('breast_cancer_data.xlsx')

df.head()

In [ ]:
# target 列は削除
df_data = df.drop('target', axis=1)

df_data

In [ ]:
# df_data の列名を feature_names とする
feature_names = df_data.columns

feature_names

In [ ]:

# NumPy配列に変換
data = df_data.to_numpy()

data

In [ ]:
# target は target としてnumpyに変換しておく
target = df['target'].to_numpy()

target

#### 5.3. データの前処理: 正規化
- 30の特徴量に対して正規化 (0-1 に変換)を行う
- 教師あり学習の時と同じ

In [ ]:
# データの正規化

# MinMaxScaler() 関数を使用して標準化を行うオブジェクト scaler を生成
scaler = MinMaxScaler()  
# 正規化するためのパラメータを計算(fit)し、適用(transform)する
data_normalized = scaler.fit_transform(data)

# 最初の数行を確認
# data, data_normalized は numpy型 なので、data[:3] で最初の3行が表示される

# data
print('data')
print(data[:3],1)

# data_normalized
print('\ndata_normalized')
print(np.round(data_normalized[:3],1))


#### 5.4. PCAの適用

- `PCA()` 関数を使用して PCAを行うオブジェクトを生成する
  - `n_components` で 圧縮する次元を指定する
- その後、準備したオブジェクトのメソッド `fit_transform()` にデータを投入する
    - `fit()`: 入力データに基づいてPCAモデルを学習
        - これにより、主成分（固有ベクトル）と、各主成分の重要度（固有値）が計算される
    - `transform()`: 学習したモデルを使って、入力データを新しい主成分空間に変換する
- `scores_pca` は、元のデータを2次元の主成分空間に投影した結果
    - 各行は元のデータに対応し、2つの列はそれぞれ第1主成分と第2主成分の値を意味する



In [ ]:
# PCAの適用

# PCAを行うオブジェクト pca を生成
pca = PCA(n_components=2)

# 前処理が終わったデータを投入し、fitで学習、transformで変換
pca_results = pca.fit_transform(data_normalized) 

- PCAの結果を確認
    - pca_resultsの形状: `pca_results.shape`
    - pca_results の実際の値 `pca_results`
    - 固有ベクトル: `pca.components_`
    - 固有値: `pca.explained_variance_`
    - 寄与率: `pca.explained_variance_ratio_`

In [ ]:
# pca_results の形状
# 569行2列
pca_results.shape

In [ ]:
# pca_results そのもの
# 小数点2位で丸め
np.round(pca_results,2)

In [ ]:
# 固有ベクトル
np.round(pca.components_.T,2)

In [ ]:
# 固有値の標準偏差

np.round(np.sqrt(pca.explained_variance_),2)

In [ ]:
# 固有値の寄与率
np.round(pca.explained_variance_ratio_,2)

In [ ]:
# 主成分負荷量

loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

np.round(loadings,2)

- 特徴量が多いと、主成分が何を意味するかすぐに把握しづらい
- こういう時に視覚化が役立つ

### 5.5. PCAの結果の視覚化

#### 5.5.1. 散布図

In [ ]:
# 散布図にプロット
# 横軸に pca_resultsの第1列、縦軸に pca_resultsの第2列をプロット

# データフレームの作成
df_pca = pd.DataFrame(data=pca_results, columns=['PC1', 'PC2'])
df_pca['diagnosis'] = target

# プロットの作成
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='diagnosis', data=df_pca, palette='winter')

plt.title('PCA of Breast Cancer Dataset')
plt.xlabel(f'First Principal Component (Variance explained: {pca.explained_variance_ratio_[0]:.2f})')
plt.ylabel(f'Second Principal Component (Variance explained: {pca.explained_variance_ratio_[1]:.2f})')

# 凡例の追加
plt.legend(title='Diagnosis', labels=['Malignant', 'Benign'])

# 軸の原点に線を追加
plt.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

#### 5.5.2. 各特徴量の寄与度

In [ ]:
# 特徴量の寄与度
plt.figure(figsize=(14, 6))

# バーの幅と位置の設定
bar_width = 0.35
r1 = np.arange(len(feature_names))
r2 = [x + bar_width for x in r1]

# 2つの主成分のloadingsを少しずらして表示
plt.bar(r1, loadings[:, 0], color='#ff7f50', width=bar_width, label='First Principal Component')
plt.bar(r2, loadings[:, 1], color='#1e90ff', width=bar_width, label='Second Principal Component')

plt.xlabel('Features')
plt.ylabel('PCA Loading')
plt.title('PCA Loadings for First and Second Principal Components')
plt.xticks([r + bar_width/2 for r in range(len(feature_names))], feature_names, rotation=90)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 第1主成分の負荷量を取得
pc1_loadings = loadings[:, 0]

# 負荷量の絶対値でソートするためのインデックスを取得
sorted_indices = np.argsort(np.abs(pc1_loadings))[::-1]

# ソートされた負荷量と対応する特徴量名を取得
sorted_loadings = pc1_loadings[sorted_indices]
sorted_features = np.array(feature_names)[sorted_indices]

# 上位10の特徴量のみを可視化
plt.figure(figsize=(10, 6))
plt.bar(range(10), sorted_loadings[:10], color='#ff7f50')
plt.xlabel('Features')
plt.ylabel('PC1 Loadings')
plt.title('Top 10 Features by PC1 Loading Magnitude')
plt.xticks(range(10), sorted_features[:10], rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 上位10の負荷量と特徴量名を表示
print("Top 10 features by PC1 loading magnitude:")
for feature, loading in list(zip(sorted_features, sorted_loadings))[:10]:
    print(f"{feature}: {loading:.2f}")

In [ ]:
# 第2主成分の負荷量を取得
pc2_loadings = loadings[:, 1]

# 負荷量の絶対値でソートするためのインデックスを取得
sorted_indices2 = np.argsort(np.abs(pc2_loadings))[::-1]

# ソートされた負荷量と対応する特徴量名を取得
sorted_loadings2 = pc2_loadings[sorted_indices2]
sorted_features2 = np.array(feature_names)[sorted_indices2]

# 上位10の特徴量のみを可視化
plt.figure(figsize=(10, 6))
plt.bar(range(10), sorted_loadings2[:10], color='#1e90ff')
plt.xlabel('Features')
plt.ylabel('PC2 Loadings')
plt.title('Top 10 Features by PC2 Loading Magnitude')
plt.xticks(range(10), sorted_features2[:10], rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 上位10の負荷量と特徴量名を表示
print("Top 10 features by PC2 loading magnitude:")
for feature, loading in list(zip(sorted_features2, sorted_loadings2))[:10]:
    print(f"{feature}: {loading:.2f}")

#### 5.6 PCA結果の医学的解釈

##### 5.6.1. 第1主成分の解釈

1. 腫瘍の形状と大きさを反映
   - 高寄与率: 凹点、凹性（形状の不規則性）
   - 高寄与率: 周囲長、半径（腫瘍の大きさ）

2. 悪性度との関連
   - "worst"特徴量の多さ: 最も異常な部分を捉える
   - 高寄与率: 密度（腫瘍の固さ）

医学的意味: 腫瘍の不規則な形状と大きさを表す。値が高いほど悪性の可能性が高い。

##### 5.6.2. 第2主成分の解釈

1. 腫瘍の微細構造と質感
   - 高正寄与率: フラクタル次元（境界線の複雑さ）
   - 高正寄与率: 滑らかさ（表面の局所的変動）

2. 大きさとの逆相関
   - 負寄与率: 半径、周囲長、面積

3. 誤差と変動性
   - 含有: 密度の誤差（測定の不確実性）

医学的意味: 腫瘍の微細構造、質感、ばらつきを表す。大きさとは独立した特徴を捉える。高値は小さいが複雑な構造の腫瘍を示唆。


### 6. k-means クラスタリング
- k-means クラスタリングは、教師なし学習の中でも最も基本的でよく使われる手法の一つ
- データセットを \( k \) 個のクラスターに分割し、各データポイントが最も近いクラスタの中心 (セントロイド) に割り当てられる

- アルゴリズムの手順
   1. 初期化
      - クラスタ数 \( k \) を決定し、データポイントからランダムに \( k \) 個のセントロイドを選ぶ
   2. 割り当て
      - 各データポイントを最も近いセントロイドに割り当てる
   3. セントロイドの更新
      - 各クラスタのセントロイドを、クラスタ内のデータポイントの平均値に更新する
   4. 収束
      - セントロイドの位置が変わらなくなるまで、割り当てと更新を繰り返す

- 評価
   - クラスタリングの質を評価するために、内部評価指標（例: シルエットスコア、クラス内分散の総和）を使用する


#### 6.1. パッケージのインポート

- NumPy
- Matplotlib
- Seaborn
- Scikit-learn の preprocessing モジュールから StandardScaler
- Scikit-learn の cluster モジュールから KMeans
- Scikit-learn の metrics モジュールから silhouette_score

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#### 6.1. データの生成
- デモンストレーションとして、明確に3つのクラスターに分かれているデータを作ったうえで、クラスタリングを行う
- scikit-learnには、クラスターを作ってくれる関数 `make_blobs` があるので、それを利用して3つのクラスターのデータを生成する

In [ ]:
from sklearn.datasets import make_blobs

# データの生成
n_samples = 120
n_clusters = 3
scores, y = make_blobs(n_samples=n_samples, centers=n_clusters, random_state=10)
scores += 70

In [ ]:
# 2つの教科の試験結果　をイメージ
np.round(scores[:5,:],0)

In [ ]:
sns.scatterplot(x=scores[:,0], y=scores[:,1])
plt.xlabel('Exam 1')
plt.ylabel('Exam 2')
plt.show()

- 明らかに3つのクラスタに分かれそう
- 3つのクラスタに分けてみる
  - クラスタの推測法は後述

#### 6.2. データの前処理: 標準化

- データの前処理を行う
- これまでと同様、StandardScaler 関数を使って標準化を行う

In [ ]:
scaler = StandardScaler()
scores_standardized = scaler.fit_transform(scores)


In [ ]:
# 標準化後のプロット
sns.scatterplot(x=scores_standardized[:,0], y=scores_standardized[:,1])
plt.xlabel('Standardized Exam 1')
plt.ylabel('Standardized Exam 2')
plt.show()


### 6.3. K-meansクラスタリング

- K-means クラスタリングでは、まず、自分で「いくつのクラスターに分類するか」を決める
- そのクラスターの数で分類器を作成する
- あとは、`fit_predict()` で学習し、予測させる
- 正確には以下のようになる
    - `fit(X)`
        - 与えられたデータXに対してk-meansアルゴリズムを実行し、クラスタ中心を学習する
        - 学習後、クラスタ中心の情報がモデルに保存されるが、各データポイントのクラスタ割り当ては返さない
    - `fit_predict(X)`
        - `fit(X)` を実行し、その後すぐに各データポイントのクラスタ割り当てを返す
        - 学習と予測を1ステップで行いたい場合に便利
    - `predict(X)`
        - 既に学習済みのモデル（`fit()`または`fit_predict()`で学習済み）に対して新しいデータXのクラスタ割り当てを予測する

In [ ]:
# K-means クラスタリングの実行
# クラスターの数を定義
n_clusters = 3

# KMeans関数で kmeans オブジェクトを生成
# 今後のセクションと混じらないように、kmeans_score とする
# n_init='auto' はデフォルト。記載しないとWarningが出るのでその抑止のため
kmeans_score = KMeans(n_init='auto', n_clusters=n_clusters, random_state=42)

# データを入れて学習させる
clusters_score = kmeans_score.fit_predict(scores_standardized)


- kmeansオブジェクトの属性から情報を抽出する
  - `cluster_centers_`: クラスタの中心 (セントロイド) の座標
  - `labels_`: 各データがどれに分類されたかを含む配列
  - `inertia_`: クラスタ内のデータとデータの中心の距離の二乗の総和
    - inertia_ が小さいほど、各データがそのクラスタの中心に近いことを意味

In [ ]:
# セントロイドの座標
np.round(kmeans_score.cluster_centers_,2)

In [ ]:
# クラスタリングで分類された結果
# 上の clusters_score = kmeans.fit_predict(scores_standardized) の clusters_scoreと同じ内容
kmeans_score.labels_

In [ ]:
# 各クラスターに所属するデータとクラスター中心の距離の二乗の総和
np.round(kmeans_score.inertia_,2)

- シルエットスコアを計算する。
  - クラスタリングの品質を評価するための指標の1つ
  - データポイントがどれだけ自身のクラスタに適合し、他のクラスタとは分離されているかを測定
  - スコアの範囲: -1から1の間の値を取る
    - 1に近い: クラスタリングが良好
    - 0に近い: クラスタ間の重なりがある
    - -1に近い: データポイントが誤ったクラスタに割り当てられている可能性が高い
- 計算方法: 各データポイントについて、(a)同じクラスタ内の他のポイントとの平均距離と、(b)最も近い他のクラスタのポイントとの平均距離を比較する


In [ ]:
# シルエットスコアを計算
silhouette = silhouette_score(scores_standardized, clusters_score)

print(f'シルエットスコア: {silhouette: .2f}')

#### 6.4. 結果の可視化

- 各クラスタを色分けし、クラスタ中心を X で示す

In [ ]:
# 結果の可視化
plt.figure(figsize=(6, 6))
plt.scatter(scores_standardized[:, 0], scores_standardized[:, 1], c=kmeans_score.labels_, cmap='winter')
plt.scatter(kmeans_score.cluster_centers_[:, 0], kmeans_score.cluster_centers_[:, 1], 
            marker='x', s=200, linewidths=3, color='r', zorder=10)
plt.title('K-means Clustering (3 clusters)')
plt.xlabel('Standardized Exam 1')
plt.ylabel('Standardized Exam 2')
plt.show()


#### 6.5. クラスター数の設定

- 上の例ではクラスター数は 3 であることが自明だが、そうはうまくいかない場合もままある
- その時に、クラスター数を1ずつ増やし、`kmeans.inertia_`を計算し、その推移を観察する
- グラフはたいていの場合、急激に減少した後に緩やかになる"肘 elbow"の形状となる
- この肘の位置が適切なクラスタ数である可能性があり、これをエルボー法という

In [ ]:
# エルボー法による適切なクラスター数の探索
inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_init='auto', n_clusters=k, random_state=42)
    kmeans.fit(scores_standardized)
    inertias.append(kmeans.inertia_)

plt.plot(k_range, inertias, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

### 7. 乳がんデータセットによるk-meansクラスタリング

- 乳がんデータセットは良性・悪性がわかっている
- 2つのクラスターにわけられるかトライする

#### 7.1. パッケージのインポート


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


#### 7.2. データの読み込みと前処理
- 今日の前半で使用した breast_cancer_data.xlsx を読み込む
- 正規化を行う

In [ ]:
# このセクションだけ実行しても大丈夫なようにファイルがない場合はダウンロードする
![[ -f breast_cancer_data.xlsx ]] || wget https://raw.githubusercontent.com/kytk/AI-MAILs/main/data/breast_cancer_data.xlsx

df = pd.read_excel('breast_cancer_data.xlsx')

# target 列は削除
df_data = df.drop('target', axis=1)

# df_data の列名を 取り出したものを feature_names とする
feature_names = df_data.columns

# NumPy配列に変換
data = df_data.to_numpy()

# target は target としてnumpyに変換しておく
target = df['target'].to_numpy()

# データの正規化

# MinMaxScaler() 関数を使用して標準化を行うオブジェクト scaler を生成
scaler = MinMaxScaler()  
# 正規化するためのパラメータを計算(fit)し、適用(transform)する
data_normalized = scaler.fit_transform(data)

In [ ]:
feature_names

#### 7.3. k-means クラスタリングの適用
- KMeans() 関数に、自分が分類したいクラスター数を指定し、kmeans という分類器を生成する
- 今は、良性と悪性で2つのクラスターができるはずという思いから、2 とする
- `kmeans.fit_predict()` で実際の分類を行う
- シルエットスコアを計算する。シルエットスコアは -1から1 の範囲で、高いほどよいクラスタリングを示す

In [ ]:
# KMeans() を使って分類器のオブジェクトを生成
kmeans_cancer = KMeans(n_init='auto', n_clusters=2, random_state=42)

# 実際にデータを入れて分類
clusters_cancer = kmeans_cancer.fit_predict(data_normalized)

# シルエットスコアを計算
silhouette = silhouette_score(data_normalized, clusters_cancer)

print(f'シルエットスコア: {silhouette: .2f}')

#### 7.4. 元の特徴量の寄与度を調べる

##### 7.4.1. クラスターの中心と特徴量の関係を調べる
- 以下の関数でクラスターの中心と特徴量の関係をグラフにできる

In [ ]:
def plot_cluster_centers(kmeans, feature_names):
    centers = kmeans.cluster_centers_
    feature_importance = np.abs(centers[0] - centers[1])
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5

    plt.figure(figsize=(12, 6))
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, np.array(feature_names)[sorted_idx])
    plt.xlabel('Absolute difference between cluster centers')
    plt.title('Feature Importance in K-means Clustering')
    plt.tight_layout()
    plt.show()

plot_cluster_centers(kmeans_cancer, feature_names)

- kmeans.cluster_centers_ を改めて見てみる

In [ ]:
# この配列の形状を確認する
kmeans_cancer.cluster_centers_.shape

In [ ]:
np.round(kmeans_cancer.cluster_centers_,2)

- これは、特徴量ごとの、クラスター1とクラスター2の中心を示している
- ある特徴量において、クラスター1とクラスター2の中心の距離が離れていたら、それは、その特徴量はクラスターを分けるのに大きく寄与しているということになる
- 今の場合、"concavity" と "perimeter" あたりのパラメーターが大きく寄与している
- "mean concavity" と "mean perimeter" のふたつでグラフを描いてみる

In [ ]:
# 'mean concavity' と 'mean perimeter' のインデックスを取得
concavity_idx = np.where(feature_names == 'mean concavity')[0][0]
perimeter_idx = np.where(feature_names == 'mean perimeter')[0][0]

# 散布図の作成
plt.figure(figsize=(8, 6))
scatter = plt.scatter(data_normalized[:, concavity_idx], data_normalized[:, perimeter_idx], 
                      c=kmeans_cancer.labels_, cmap='winter', alpha=0.7)

plt.xlabel('Mean Concavity (standardized)')
plt.ylabel('Mean Perimeter (standardized)')
plt.title('Scatter Plot: Mean Concavity vs Mean Perimeter')

# クラスターの中心をプロット
centers = kmeans_cancer.cluster_centers_
plt.scatter(centers[:, concavity_idx], centers[:, perimeter_idx], 
            c='red', s=200, alpha=0.8, marker='X', label='Cluster Centers')

plt.legend()
plt.colorbar(scatter, label='Cluster')
plt.tight_layout()
plt.show()

# クラスターごとの統計情報を表示
for i in range(2):
    cluster_points = data_normalized[kmeans_cancer.labels_ == i]
    print(f"Cluster {i}:")
    print(f"  Mean Concavity: {np.mean(cluster_points[:, concavity_idx]):.2f}")
    print(f"  Mean Perimeter: {np.mean(cluster_points[:, perimeter_idx]):.2f}")
    print(f"  Number of points: {len(cluster_points)}")
    print()

#### 7.5. 良性、悪性とクラスター分割の一致
- kmeans クラスタリングの結果と良性、悪性の診断がどの程度一致するかを見てみる
- kmeans クラスタリングはあくまでもクラスタリングであり、良性、悪性にわけるためのものではないことに注意
- 調整ランド指数 (adjusted rand index; ARI) を使用する
    - ARIは-1から1の範囲を取り、1に近いほどクラスタリング結果が実際のラベルと一致していることを示す
    - 0は偶然の一致を意味し、負の値は偶然より悪い結果を示す

In [ ]:
# sklearn.metrics から adjusted_rand_score をインポート
from sklearn.metrics import adjusted_rand_score

# 良性、悪性の値が入っている target と
# 今回のクラスタリングの結果が入っている clusters を比較する
ari = adjusted_rand_score(target, clusters_cancer)
print(f"\n調整ランド指数（ARI）: {ari:.2f}")

#### 7.6. 乳がんデータセットのクラスター数の探索

- 乳がんデータセットは良性か悪性かの2つに分かれているが、実際はクラスターはもう少しあるかもしれない
- エルボー法で探索してみる

In [ ]:
# エルボー法による適切なクラスター数の探索
inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans_cancer_explore = KMeans(n_init='auto', n_clusters=k, random_state=42)
    kmeans_cancer_explore.fit(data_normalized)
    inertias.append(kmeans_cancer_explore.inertia_)

plt.plot(k_range, inertias, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

- 2 or 3 程度と考えられる

### 7.7 K-meansクラスタリングのクロスバリデーション

- クロスバリデーションは通常、教師あり学習で使用されるが、K-meansクラスタリングのような教師なし学習にも応用できる
- クロスバリデーションの目的: クラスタリング結果の安定性と信頼性を評価する

#### K-meansクラスタリングでのクロスバリデーションの手順：

1. データセットを複数の部分集合（フォールド）に分割する。
2. 各フォールドを順番にテストセットとして使用し、残りをトレーニングセットとする。
3. トレーニングセットでK-meansを実行し、モデルを構築する。
4. 構築したモデルをテストセットに適用し、クラスタリング結果を評価する。
5. 全フォールドで手順3-4を繰り返し、結果の平均を取る。

- 評価指標としては、シルエットスコアやクラスタ内分散などを使用できる。これにより、クラスタリング結果が特定のデータ分割に依存せず、一貫性があることを確認できる

- 注意点：K-meansは初期中心点によって結果が変わる可能性があるため、各フォールドで複数回実行し、最良の結果を使用することも検討する

In [ ]:
# クロスバリデーションを実行
# 視覚化するために、まず、PCAを実行し、2次元に次元削減する
# その後、データを5分割し、クロスバリデーションを行う


import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

def kmeans_cv_visualization(data, n_clusters=2, n_splits=5, random_state=42):
    # KFoldでデータを分割するためのオブジェクト kf を生成する
    #  n_splits: 分割する個数
    #  shuffle=True: 分割する前にデータをシャッフルする
    #  random_state: シャッフルする際の乱数表をどれを使うか
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'K-means Clustering Cross-Validation and Whole data (n_clusters={n_clusters})', fontsize=16)

    # PCAを適用してデータを2次元に削減
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data)

    # kf.split(data) で、データのインデックスが分割される
    # enumerate() 関数は、分割の反復回数（0から始まる）と、
    # 各分割で得られる（train_index, test_index）のタプルを返す
    # 分割の反復回数が for の中の変数 i に代入される
    # 今回は5分割されるので、5分の4がモデルの学習に使われ、
    # 5分の1のテストデータに対し、モデルが適用される
    for i, (train_index, test_index) in enumerate(kf.split(data)):
        if i < 5:  # 最初の5つのサブプロットはクロスバリデーション用
            # KMeansのオブジェクトを生成
            kfold_kmeans = KMeans(n_init='auto', n_clusters=n_clusters, random_state=random_state)
            # 訓練データに対して、KMeansでモデルを学習
            kfold_kmeans.fit(data[train_index])
            # テストデータに対して、モデルを適用し、クラスターを予測
            test_labels = kfold_kmeans.predict(data[test_index])

            # テストデータのプロット
            ax = axs[i//3, i%3]
            scatter = ax.scatter(data_2d[test_index, 0], data_2d[test_index, 1], c=test_labels, cmap='winter')
            ax.set_title(f'Fold {i+1}')
            ax.set_xlabel('First Principal Component')
            ax.set_ylabel('Second Principal Component')
            ax.set_xlim(-1.2,2.5)
            ax.set_ylim(-1.2,1.5)

            # クラスターの中心をプロット
            centers_2d = pca.transform(kfold_kmeans.cluster_centers_)
            ax.scatter(centers_2d[:, 0], centers_2d[:, 1], c='red', marker='x', s=200, linewidths=3)

            # シルエットスコアを計算
            score = silhouette_score(data[test_index], test_labels)
            ax.text(0.05, 0.95, f'Silhouette Score: {score:.3f}', transform=ax.transAxes, verticalalignment='top')

    # 6番目のサブプロットに全データを使用した時のクラスタリング結果を表示
    ax = axs[1, 2]
    whole_kmeans = KMeans(n_init='auto', n_clusters=n_clusters, random_state=random_state)
    whole_labels = whole_kmeans.fit_predict(data)
    scatter = ax.scatter(data_2d[:, 0], data_2d[:, 1], c=whole_labels, cmap='winter')
    ax.set_title('Whole Data')
    ax.set_xlabel('First Principal Component')
    ax.set_ylabel('Second Principal Component')
    ax.set_xlim(-1.2,2.5)
    ax.set_ylim(-1.2,1.5)


    # 全データを使った時のクラスターの中心をプロット
    whole_centers_2d = pca.transform(whole_kmeans.cluster_centers_)
    ax.scatter(whole_centers_2d[:, 0], whole_centers_2d[:, 1], c='red', marker='x', s=200, linewidths=3)


    # 全データを使った時のシルエットスコアを計算
    whole_score = silhouette_score(data, whole_labels)
    ax.text(0.05, 0.95, f'Silhouette Score: {whole_score:.3f}', transform=ax.transAxes, verticalalignment='top')

    plt.tight_layout()
    plt.show()

    return whole_score, whole_labels

# n_clusters=2 でのクロスバリデーションと全データのクラスタリングの視覚化
whole_score, whole_labels = kmeans_cv_visualization(data_normalized, n_clusters=2)


- クロスバリデーションと全データを使ったクラスタリングの結果の解釈:
    - 最初の5つのサブプロットは、5分割交差検証の各フォールドでのテストデータのクラスタリング結果
    - 6番目のサブプロットは、全データを使用したクラスタリング結果
    - 点の色はクラスターの割り当てを、赤い×印はクラスターの中心
    - 各プロットのシルエットスコアにより、分類の質を評価できる
    - クロスバリデーションの結果と全データの結果を比較することで、モデルの安定性と一般化能力を評価できる
    - 全データのシルエットスコア（0.385）は、モデル全体の性能を示している
    - クラスターが明確に分離されているほど、そのクラスター数がデータに適していることを示唆している


#### クロスバリデーションを応用したクラスター数の推測
- クロスバリデーションで得られるシルエットスコアから平均と標準偏差をとって、最適なクラスター数を推測する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.metrics import silhouette_score

def kmeans_cross_validation(data, n_clusters, n_splits=5, n_init=10, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    silhouette_scores = []

    for train_index, test_index in kf.split(data):
        kmeans = KMeans(n_clusters=n_clusters, n_init=n_init, random_state=random_state)
        kmeans.fit(data[train_index])
        test_labels = kmeans.predict(data[test_index])
        score = silhouette_score(data[test_index], test_labels)
        silhouette_scores.append(score)

    return np.mean(silhouette_scores), np.std(silhouette_scores)

# クラスター数の範囲を設定
cluster_range = range(2, 11)

# 各クラスター数でクロスバリデーションを実行
cv_scores = []
for n_clusters in cluster_range:
    mean_score, std_score = kmeans_cross_validation(data_normalized, n_clusters)
    cv_scores.append((mean_score, std_score))
    print(f"Number of clusters {n_clusters}: Mean silhouette score = {mean_score:.3f} (±{std_score:.3f})")

# 結果をプロット
means, stds = zip(*cv_scores)
plt.figure(figsize=(6, 4))
plt.errorbar(cluster_range, means, yerr=stds, fmt='o-', capsize=5)
plt.xlabel('Number of Clusters')
plt.ylabel('Mean Silhouette Score')
plt.title('K-means Clustering Cross-Validation Results')
plt.show()

# 最適なクラスター数を特定
# 平均シルエットスコアが最大のものを選択
optimal_clusters = cluster_range[np.argmax(means)]
print(f"\nOptimal number of clusters: {optimal_clusters}")
print(f"Highest mean silhouette score: {max(means):.3f}")


- 結果の解釈
    - シルエットスコアが最も高いクラスター数が、データに最適なクラスター数と考えられる
    - エラーバーは各クラスター数での結果の安定性を示している。エラーバーが小さいほど、結果が安定している
    - クラスター数が増えるにつれてスコアが低下する場合、クラスター数は小さいことを示唆している
    - 最適なクラスター数が2に近い場合、これは元々の良性/悪性の二分類と一致する可能性がある
    - ただし、クラスター数の選択は常に問題の文脈や目的に応じて行う必要がある

----

### おわりに
- 以上、Pythonを使った医療データの解析の基本を一通りカバーしました
  - Pythonの基礎
  - Pandas, NumPy, Seaborn, Matplotlib
  - 機械学習
  - 深層学習
- 私自身、学びながらのところもたくさんありましたが、基本をおさえることで、応用しやすくなると思います
- 皆様が少しでもこのような領域に興味・関心を持っていただけたら、準備した甲斐があります
- 様々なご意見もありがとうございました。最後の感想もぜひお聞かせください


##### (参考) 主成分分析で使用したデータの生成方法

- 150人の仮想の試験結果を生成
- 理系科目が得意な75人と文系科目が得意な75人を想定
- 5教科 (英語、数学、理科、国語、社会) を乱数で40-75点で生成
- 全員に対して、英語は乱数で5-20点をかさ増し
- 理系科目が得意な75人に対して、数学、理科を5-20点かさ増し
- 文系科目が得意な75人に対して、国語、社会を5-20点かさ増し



In [ ]:
# 主成分分析で使用したテストの成績の生成

# 乱数のシードを設定して再現性を確保
np.random.seed(42)

# 初期化
scores = []

# 生徒の数
n_students = 150

# 理系群と文系群の生徒数（ほぼ半々に）
# 理系群は生徒の数を2で割った時の商
# 文系群は 全生徒の数 - 理系生徒の数
n_science = n_students // 2
n_humanities = n_students - n_science

# 基本的な成績の生成（全科目）
scores = np.random.randint(40, 76, size=(n_students, 5))

# 英語の成績調整
scores[:, 0] += np.random.randint(5, 20, n_students)

# 理系群の成績調整
scores[:n_science, 1] += np.random.randint(5, 20, n_science)  # 数学
scores[:n_science, 2] += np.random.randint(5, 20, n_science)  # 理科

# 文系群の成績調整
scores[n_science:, 3] += np.random.randint(5, 20, n_humanities)  # 国語
scores[n_science:, 4] += np.random.randint(5, 20, n_humanities)  # 社会

# データフレームの作成
df = pd.DataFrame(scores, columns=['English', 'Math', 'Science', 'Japanese', 'Social_studies'])

# 実際の群（理系/文系）のラベルを追加
df['group'] = ['science' if i < n_science else 'humanities' for i in range(n_students)]

# データの保存
df.to_csv('student_scores.csv', index=False)